In [ ]:
source("../tools/formating/formating.R")

In [ ]:
path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"

In [ ]:
srat <- LoadSeurat(path)
srat

In [ ]:
default_assay <- DefaultAssay(srat)
default_assay

In [ ]:
srat[['RNA']]@counts

In [ ]:
dim(srat[['RNA']]@counts)
dim(srat)
dim(srat@meta.data)

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
head(srat@meta.data)

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)

In [ ]:
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat1@meta.data

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
# Calculate the percentage of mitocondrial per cell and add to the metadata.
if(! "percent.mt" %in% names(x = srat[[]])) srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")
# Calculate the proportion gene expression that comes from ribosomal proteins.
if(! "percent.rb" %in% names(x = srat[[]])) srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]")

# Percentage hemoglobin genes - includes all genes starting with HB except HBP.
if(! "percent.hb" %in% names(x = srat[[]])) srat[["percent.hb"]] <- PercentageFeatureSet(srat, pattern = "^HB[^(P)]")
if(! "percent.plat" %in% names(x = srat[[]])) srat[["percent.plat"]] <- PercentageFeatureSet(srat, pattern = "PECAM1|PF4")
head(srat@meta.data)

In [ ]:
# Add the doublet annotation
doublets <- read.table(here::here('api/tools/qc/scrublet_calls.tsv'), header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
# srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
head(srat@meta.data)

In [ ]:
srat@meta.data

In [ ]:
nFeature_min=200
nFeature_max=0
percent_mt_max=5
percent_rb_min=0
srat <- subset(srat, subset = paste0("nFeature_", default_assay) > nFeature_min & percent.mt < percent_mt_max)
if(nFeature_max != 0) srat <- subset(srat, subset = paste0("nFeature_", default_assay) < nFeature_max)
if(percent_rb_min != 0)  srat <- subset(srat, subset = percent.rb > percent_rb_min)

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True' | is.na(Is_doublet))

In [ ]:
#' Regularise dataframe
#'
#' This function checks if certain columns of a dataframe is of a single value
#' and drop them if required
#'
#' @param df Input data frame, usually cell metadata table (data.frame-like
#'   object)
#' @param drop_single_values Drop columns with only a single value (logical)
#'
#' @return Dataframe
.regularise_df <- function(df, drop_single_values=FALSE, drop_na_values=TRUE) {
  if (ncol(df) == 0) df[["name"]] <- rownames(df)
  if (drop_single_values) {
    k_singular <- sapply(df, function(x) length(unique(x)) == 1)
    if (sum(k_singular) > 0) {
      warning(
        paste("Dropping single category variables:"),
        paste(colnames(df)[k_singular], collapse = ", ")
      )
    }
    df <- df[, !k_singular, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
 if (drop_na_values) {
    k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
    if (sum(k_na) > 0) {
      warning(
        paste("Dropping NA category variables:"),
        paste(colnames(df)[k_na], collapse = ", ")
      )
    }
    df <- df[, !k_na, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
  return(df)
}

In [ ]:
output_path_rds <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds'
saveRDS(srat, output_path_rds)
srat1 <- LoadSeurat(output_path_rds)
srat1

In [ ]:
k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
k_na
sum(k_na)

In [ ]:
srat_qc@meta.data %>% is.na()

In [ ]:
k_singular <- sapply(srat_qc@meta.data, function(x) length(unique(x)) == 1)
k_singular

In [ ]:
srat_qc@meta.data

In [ ]:
srat@meta.data <- .regularise_df(srat@meta.data, drop_single_values=FALSE, drop_na_values=TRUE)
srat@meta.data

In [ ]:
srat_qc@meta.data <- .regularise_df(srat_qc@meta.data, drop_single_values = FALSE)
srat_qc@meta.data

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_3 <- LoadH5Seurat(output_path)
srat_3

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat1, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_2 <- LoadH5Seurat(output_path)
srat_2

In [ ]:
srat1[["meta.data"]]

In [ ]:
srat1@meta.data

In [ ]:
devtools::install_github("cellgeni/sceasy")

In [ ]:
sceasy::convertFormat(srat1, from="seurat", to="anndata",
                       outFile='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')

In [ ]:
adata <- LoadAnndata('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')
adata

In [ ]:
adata$obs

In [ ]:
adata$X

In [ ]:
srat_qc <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data, project = Project(srat))
srat_qc

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_qc_1 <- LoadH5Seurat(output_path)
srat_qc_1

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')

In [ ]:
srat[["meta.data"]]

In [ ]:
Cells(srat)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))
srat
dim(srat)
dim(srat@meta.data)

In [ ]:
srat[['Doublet_score']][sapply(srat[['Doublet_score']], is.na)] <- 0
srat[['Doublet_score']]

In [ ]:
srat[['Doublet_score']] <- !is.na(srat[['Doublet_score']])


In [ ]:
# srat <- subset(srat, subset = percent.rb >= 0)
srat <- subset(srat, subset = is.na(Is_doublet))
srat

In [ ]:
srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')
srat

In [ ]:
srat@meta.data

In [ ]:
'nCount_RNA' %in% names(srat@meta.data)

In [ ]:
result <- GetMetadataFromSeurat(path)
result

In [ ]:
suffix <- tolower(get_suffix(path))
suffix

In [ ]:
seurat_object <- LoadSeurat(path)
seurat_object

In [ ]:
'umap' %in% names(seurat_object@reductions)

In [ ]:
genes <- rownames(seurat_object)
typeof(genes)
genes

In [ ]:
cells <- Cells(seurat_object)
typeof(cells)
cells

In [ ]:
pca <- Embeddings(object = seurat_object, reduction = "pca")
typeof(pca)
pca

In [ ]:
name <- load(path)
seurat_object <- UpdateSeuratObject(get(name))

In [ ]:
seurat_object

In [ ]:
features <- seurat_object[['RNA']]@var.features
typeof(features)
features

In [ ]:
#获取当前用的Idents
Idents(seurat_object)
levels(seurat_object)

In [ ]:
#总数目
nCells <- ncol(seurat_object)
nGenes <- nrow(seurat_object)

nCells
nGenes

In [ ]:
dim=1:10
class(dim)

In [ ]:
x <- 1:10
x

In [ ]:
class(x)